# 0 Подготовка к работе

In [10]:
# Библиотека для работы с HTTP-запросами. Будем использовать ее для обращения к API HH
import requests
 
# Пакет для удобной работы с данными в формате json
import json
 
# Модуль для работы со значением времени
import time
 
# Модуль для работы с операционной системой. Будем использовать для работы с файлами
import os

import pandas as pd #библиотека для работы с бд
import numpy as np
import glob

# 1 Парсинг

In [11]:
def getPage(page=0):
    """
    Получение страницы со списком вакансий.
    Аргументы:
        page - Индекс страницы, начинается с 0. Значение по умолчанию 0, т.е. первая страница
    """
     
    # Справочник для параметров GET-запроса
    params = {
        'text':'NAME:""',          # Текст фильтра. В имени должно быть определенное слово 
        'area': 113,               # Поиск ощуществляется по России
        'page': page,              # Индекс страницы поиска на HH
        'per_page': 100,           # Кол-во вакансий на 1 странице
        'specialization' : 1,      # Информационные технологии
        'profarea_id' : 1,
        'date_from' : '2021-11-15',
        'date_to' : '2021-11-16'
    }
    
    # Посылаем запрос к API
    req = requests.get('https://api.hh.ru/vacancies', params)     
    # Декодируем его ответ, чтобы Кириллица отображалась корректно
    data = req.content.decode() 
    req.close()
    
    return data

In [12]:
jsObjs = [] 

# Считываем первые 2000 вакансий
for page in range(0, 20):
     
    # Преобразуем текст ответа запроса в справочник Python
    jsObj = json.loads(getPage(page))
    
    # добавление текущего ответа запроса в список
    jsObjs.extend(jsObj["items"]) 
    
    # Сохраняем файлы в папку {путь до текущего документа со скриптом}\docs\pagination
    # Определяем количество файлов в папке для сохранения документа с ответом запроса
    # Полученное значение используем для формирования имени документа
    nextFileName = './parsing/{}.json'.format(len(os.listdir('./parsing')))
     
    # Создаем новый документ, записываем в него ответ запроса, после закрываем
    f = open(nextFileName, mode='w', encoding='utf8')
    f.write(json.dumps(jsObj, ensure_ascii=False))
    f.close()
     
    # Проверка на последнюю страницу, если вакансий меньше 2000
    if (jsObj['pages'] - page) <= 1:
        break
     
    # Необязательная задержка, но чтобы не нагружать сервисы hh, оставим.
    time.sleep(0.25)

print('Страницы поиска собраны')

Страницы поиска собраны


In [13]:
# Получаем карточки вакансий
v = 0

for v in jsObjs: #цикл по справочнику
         
    # Обращаемся к API и получаем детальную информацию по конкретной вакансии
    req = requests.get(v['url'])
    data = req.content.decode()
    req.close()
        
    # Создаем файл json с идентификатором вакансии в качестве названия
    fileName = './vacancies/{}.json'.format(v['id'])
    # Записываем в него ответ запроса и закрываем
    f = open(fileName, mode='w', encoding='utf8')
    f.write(data)
    f.close()
         
    time.sleep(0.25)
         
print('Вакансии собраны')

Вакансии собраны


In [ ]:
## df = pd.DataFrame(jsObjs)
#df.head(2)

# 2 Предварительный обзор и предобработка данных

In [14]:
# Модуль для работы с отображением вывода Jupyter
from IPython import display

skills_vac = []    # Список идентификаторов вакансий
skills_name = []   # Список названий навыков
 
# В выводе будем отображать прогресс
# Для этого узнаем общее количество файлов, которые надо обработать
cnt_docs = len(os.listdir('./vacancies'))
i = 0  # Счетчик обработанных файлов установим в ноль

df_vac = pd.DataFrame([])

# Проходимся по всем файлам в папке vacancies
for fl in os.listdir('./vacancies'): 
            
    # Открываем, читаем и закрываем файл
    f = open('./vacancies/{}'.format(fl), encoding='utf8')
    jsonText = f.read()
    f.close()
     
    # Текст файла переводим в справочник
    jsonObj = json.loads(jsonText)
    # Добавляем строки в датафрейм
    df_vac = df_vac.append(pd.DataFrame.from_dict(pd.json_normalize(jsonObj), orient='columns')) 
        
    # Увеличиваем счетчик обработанных файлов на 1, очищаем вывод ячейки и выводим прогресс
    i += 1
    display.clear_output(wait=True)
    display.display('Готово {} из {}'.format(i, cnt_docs))

In [4]:
df_vac.drop(['address.metro.station_id','insider_interview.id', 'insider_interview.url', 'vacancy_constructor_template.bottom_picture.path', 'vacancy_constructor_template.bottom_picture.blurred_path', 'vacancy_constructor_template.bottom_picture.height', 'vacancy_constructor_template.top_picture.width', 'vacancy_constructor_template.top_picture.path', 'vacancy_constructor_template.top_picture.blurred_path', 'vacancy_constructor_template','salary.gross', 'address.metro', 'address', 'test.required', 'employer.logo_urls', 'vacancy_constructor_template.id', 'vacancy_constructor_template.name','employer.logo_urls.240', 'employer.logo_urls.90', 'employer.logo_urls.original', 'employer.vacancies_url', 'employer.trusted', 'department.id', 'department.name','site.id', 'site.name', 'experience.id', 'schedule.id', 'employment.id', 'employer.id', 'employer.name', 'employer.url', 'employer.alternate_url','type.id', 'type.name', 'address.city', 'address.street', 'address.building', 'address.description', 'address.raw', 'address.metro.station_name', 'address.metro.line_name', 'address.metro.line_id', 'address.metro.lat', 'address.metro.lng', 'address.metro_stations','billing_type.name', 'area.url','apply_alternate_url', 'alternate_url', 'working_days', 'working_time_intervals', 'working_time_modes', 'accept_temporary', 'billing_type.id','created_at', 'negotiations_url', 'suitable_resumes_url', 'has_test', 'test', 'premium', 'relations', 'insider_interview','response_letter_required', 'salary', 'allow_messages','department', 'contacts', 'branded_description', 'vacancy_constructor_template.top_picture.height', 'accept_handicapped', 'accept_kids', 'archived', 'response_url', 'code', 'hidden', 'quick_responses_allowed', 'driver_license_types', 'accept_incomplete_resumes'], axis=1, inplace=True)


In [8]:
df_vac.head(2)

,id,name,description,key_skills,specializations,professional_roles,area.id,area.name,salary.from,salary.to,salary.currency,address.lat,address.lng,experience.name,schedule.name,employment.name
0,14694668,Junior разработчик (стажер),<p><strong>Должностные обязанности:</strong></...,[],"[{'id': '1.172', 'name': 'Начальный уровень, М...","[{'id': '96', 'name': 'Программист, разработчи...",92,Тула,20000.0,None,RUR,54.185099,37.612012,Нет опыта,Полный день,Полная занятость
0,14820007,Программист 1С,"<p><strong>СофтЭксперт - IT-интегратор, крупне...","[{'name': '1С программирование'}, {'name': '1С...","[{'id': '1.221', 'name': 'Программирование, Ра...","[{'id': '96', 'name': 'Программист, разработчи...",92,Тула,80000.0,None,RUR,54.18209,37.573411,От 1 года до 3 лет,Удаленная работа,Полная занятость


In [15]:
# df_vac.drop([0], axis=1, inplace=True)

In [7]:
df_vac.drop(['published_at', 'immediate_redirect_url'], axis=1, inplace=True)

In [9]:
df_vac.to_csv('20211122_BAZA_IT.csv')

In [3]:
data = pd.read_csv("20211122_BAZA_IT.csv")
data.head(2)

,Unnamed: 0,id,name,description,key_skills,specializations,professional_roles,area.id,area.name,salary.from,salary.to,salary.currency,address.lat,address.lng,experience.name,schedule.name,employment.name
0,0,14694668,Junior разработчик (стажер),<p><strong>Должностные обязанности:</strong></...,[],"[{'id': '1.172', 'name': 'Начальный уровень, М...","[{'id': '96', 'name': 'Программист, разработчи...",92,Тула,20000.0,NaN,RUR,54.185099,37.612012,Нет опыта,Полный день,Полная занятость
1,0,14820007,Программист 1С,"<p><strong>СофтЭксперт - IT-интегратор, крупне...","[{'name': '1С программирование'}, {'name': '1С...","[{'id': '1.221', 'name': 'Программирование, Ра...","[{'id': '96', 'name': 'Программист, разработчи...",92,Тула,80000.0,NaN,RUR,54.182090,37.573411,От 1 года до 3 лет,Удаленная работа,Полная занятость


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56577 entries, 0 to 56576
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          56577 non-null  int64  
 1   id                  56577 non-null  int64  
 2   name                56577 non-null  object 
 3   description         56577 non-null  object 
 4   key_skills          56577 non-null  object 
 5   specializations     56577 non-null  object 
 6   professional_roles  56577 non-null  object 
 7   area.id             56577 non-null  int64  
 8   area.name           56577 non-null  object 
 9   salary.from         29649 non-null  float64
 10  salary.to           16855 non-null  float64
 11  salary.currency     32459 non-null  object 
 12  address.lat         25612 non-null  float64
 13  address.lng         25612 non-null  float64
 14  experience.name     56577 non-null  object 
 15  schedule.name       56577 non-null  object 
 16  empl

In [7]:
data['experience.name'].value_counts()

От 1 года до 3 лет    25946
Нет опыта             17927
От 3 до 6 лет         11704
Более 6 лет            1000
Name: experience.name, dtype: int64

In [8]:
data['professional_roles'].value_counts()

[{'id': '96', 'name': 'Программист, разработчик'}]                                12681
[{'id': '40', 'name': 'Другое'}]                                                   7760
[{'id': '97', 'name': 'Продавец-консультант, продавец-кассир'}]                    3889
[{'id': '121', 'name': 'Специалист технической поддержки'}]                        3651
[{'id': '70', 'name': 'Менеджер по продажам, менеджер по работе с клиентами'}]     3393
                                                                                  ...  
[{'id': '135', 'name': 'Финансовый директор (CFO)'}]                                  1
[{'id': '63', 'name': 'Машинист'}]                                                    1
[{'id': '95', 'name': 'Полицейский'}]                                                 1
[{'id': '136', 'name': 'Финансовый контролер'}]                                       1
[{'id': '61', 'name': 'Мастер ногтевого сервиса'}]                                    1
Name: professional_roles, Length

In [10]:
data['area.name'].value_counts()

Москва                         18647
Санкт-Петербург                 3803
Новосибирск                     3529
Казань                          2526
Воронеж                         2124
                               ...  
Заводской (Северная Осетия)        1
Выльгорт                           1
Ивня                               1
Вынгапуровский                     1
Оршанка                            1
Name: area.name, Length: 1015, dtype: int64

In [11]:
data['specializations'].value_counts()

[{'id': '1.221', 'name': 'Программирование, Разработка', 'profarea_id': '1', 'profarea_name': 'Информационные технологии, интернет, телеком'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   3953
[{'id': '17.256', 'name': 'Розничная торговля', 'profarea_id': '17', 'profarea_name': 'Продажи'}, {'id': '17.535', 'name': 'Торговые сети', 'profarea_id': '17', 'profarea_name': 'Продажи'}, {'id

In [12]:
df = pd.read_csv("20211122_analitik.csv")
df.head(2)

,Unnamed: 0,id,name,description,key_skills,specializations,professional_roles,published_at,area.id,area.name,address.lat,address.lng,experience.name,schedule.name,employment.name,salary.from,salary.to,salary.currency
0,0,24720534,Системный/бизнес-аналитик,<p><strong>В компанию АО </strong> <strong>«НП...,[],"[{'id': '12.251', 'name': 'Реинжиниринг бизнес...","[{'id': '10', 'name': 'Аналитик'}]",2021-11-08T10:03:32+0300,1,Москва,NaN,NaN,От 1 года до 3 лет,Полный день,Полная занятость,NaN,NaN,NaN
1,0,30185978,Системный аналитик,<p><strong>Мы делаем автомобили доступными.</s...,[],"[{'id': '1.221', 'name': 'Программирование, Ра...","[{'id': '10', 'name': 'Аналитик'}]",2021-11-20T10:11:47+0300,1,Москва,55.728849,37.620321,От 1 года до 3 лет,Удаленная работа,Полная занятость,NaN,NaN,NaN


In [13]:
new_df = pd.concat([df, data])

In [14]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 65262 entries, 0 to 56576
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          65262 non-null  int64  
 1   id                  65262 non-null  int64  
 2   name                65262 non-null  object 
 3   description         65262 non-null  object 
 4   key_skills          65262 non-null  object 
 5   specializations     65262 non-null  object 
 6   professional_roles  65262 non-null  object 
 7   published_at        8685 non-null   object 
 8   area.id             65262 non-null  int64  
 9   area.name           65262 non-null  object 
 10  address.lat         29921 non-null  float64
 11  address.lng         29921 non-null  float64
 12  experience.name     65262 non-null  object 
 13  schedule.name       65262 non-null  object 
 14  employment.name     65262 non-null  object 
 15  salary.from         31401 non-null  float64
 16  sala

In [19]:
new_df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [20]:
new_df.head(2)

,id,name,description,key_skills,specializations,professional_roles,published_at,area.id,area.name,address.lat,address.lng,experience.name,schedule.name,employment.name,salary.from,salary.to,salary.currency
0,24720534,Системный/бизнес-аналитик,<p><strong>В компанию АО </strong> <strong>«НП...,[],"[{'id': '12.251', 'name': 'Реинжиниринг бизнес...","[{'id': '10', 'name': 'Аналитик'}]",2021-11-08T10:03:32+0300,1,Москва,NaN,NaN,От 1 года до 3 лет,Полный день,Полная занятость,NaN,NaN,NaN
1,30185978,Системный аналитик,<p><strong>Мы делаем автомобили доступными.</s...,[],"[{'id': '1.221', 'name': 'Программирование, Ра...","[{'id': '10', 'name': 'Аналитик'}]",2021-11-20T10:11:47+0300,1,Москва,55.728849,37.620321,От 1 года до 3 лет,Удаленная работа,Полная занятость,NaN,NaN,NaN


In [21]:
new_df.drop_duplicates

<bound method DataFrame.drop_duplicates of              id                                       name  \
0      24720534                  Системный/бизнес-аналитик   
1      30185978                         Системный аналитик   
2      30649371            Инженер-аналитик (контент SIEM)   
3      30964298              Product Owner (Telephony API)   
4      31069427  Системный аналитик (Тинькофф Страхование)   
...         ...                                        ...   
56572  49805851        Разработчик React Native - удаленно   
56573  49805852               Младший программист - стажер   
56574  49806000                 Аналитик отдела продаж CRM   
56575  49807345                                   Продавец   
56576   6555939                     Ведущий программист 1С   

                                             description  \
0      <p><strong>В компанию АО </strong> <strong>«НП...   
1      <p><strong>Мы делаем автомобили доступными.</s...   
2      <p><strong>Вам предстоит:

In [22]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 65262 entries, 0 to 56576
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  65262 non-null  int64  
 1   name                65262 non-null  object 
 2   description         65262 non-null  object 
 3   key_skills          65262 non-null  object 
 4   specializations     65262 non-null  object 
 5   professional_roles  65262 non-null  object 
 6   published_at        8685 non-null   object 
 7   area.id             65262 non-null  int64  
 8   area.name           65262 non-null  object 
 9   address.lat         29921 non-null  float64
 10  address.lng         29921 non-null  float64
 11  experience.name     65262 non-null  object 
 12  schedule.name       65262 non-null  object 
 13  employment.name     65262 non-null  object 
 14  salary.from         31401 non-null  float64
 15  salary.to           18156 non-null  float64
 16  sala

In [23]:
new_df.to_csv('Final.csv')